In [ ]:
# 이미지있으면 이미지처리 -> 관련키워드는 따로
# 자격요건 넣기
# 랜덤으로 timesleep 주기
# 막혔을때 예외처리
# 스킬 추가

In [34]:
# conda install beautifulsoup4
# conda install selenium
import time
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import random
import re

In [69]:
def remove_tags(string):
    return re.sub('<.+?>', '', str(string), 0).strip()

Data = pd.DataFrame({}, columns=[\
    'title','job', 'cname','duties','skills', 'preference', 'welfare', 'loc',\
    'rec type','url'])

In [5]:
options = webdriver.ChromeOptions()
#options.add_argument('--headless')        ## Head-less 설정
#options.add_argument('--no-sandbox')
#options.add_argumetn('--blink-setting=imagesEnabled=false') # 이미지 로딩 안함
#options.add_argument('--nute-audio') # 음소거
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-site-isolation-trials")
#options.add_argument('--disable-setuid-sandbox')
options.add_argument('--incognito') # 시크릿모드
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36') # user와 같은 설정 주기
chromedriver = '../data/chromedriver'

In [7]:
driver.quit()
driver = webdriver.Chrome(chromedriver, options=options) # 크롬 드라이버 설정

In [8]:
base_url = 'https://www.jobkorea.co.kr/'
driver.get(base_url)
search_job = driver.find_element('xpath', '//*[@id="stext"]')
word = '데이터 사이언티스트'
search_job.send_keys(word+Keys.ENTER)
time.sleep(5) # 랜덤으로
html = driver.page_source

In [120]:
page_soup = BeautifulSoup(html, 'html.parser')
recruit_items = page_soup.find_all('li', {'class':'list-post'}) # list-post / list-post active

In [159]:
####### 각 채용공고당 진행
# for-loop start
import sys
import copy

#for recruit in recruits:
recruits = recruit_items[2].find('a', {'class':'title dev_view'})
rec_url = recruits['href']    
base_url = 'https://www.jobkorea.co.kr'
search_url = base_url + rec_url
print (search_url)
#r = requests.get(search_url)
driver.get(search_url)
stime = 3*random.random()
time.sleep(stime)
html_job = BeautifulSoup(driver.page_source, 'html.parser')

https://www.jobkorea.co.kr/Recruit/GI_Read/38870020?Oem_Code=C1&logpath=1&stext=데이터 사이언티스트&listno=3


In [160]:
iframe = html_job.find('iframe', {'title':'상세요강'}) # html 내의 document (iframe) 처리
driver.switch_to.frame('gib_frame') # iframe으로 옮겨가기
stime = 3*random.random()
time.sleep(stime)
html_iframe = BeautifulSoup(driver.page_source, 'html.parser')

In [161]:
title = remove_tags(html_job.find('title'))
job = word
company = remove_tags(html_job.find('span', {'class':'coName'}))
html_str = str(html_iframe)
url = search_url
print ('title : ', title)
print ('job : ', job)
print ('company : ', company)
print ('url : ', url)

# keywords and location
keyword = ''
loc = ''
assoc_keyword = html_job.find('article', {'id':'artKeywordSearch'})
for key in assoc_keyword.find_all('li'):
    if str(key).find('btnLocal') > -1:
        loc = remove_tags(key).replace('&gt;', '>')
        continue
    keyword += remove_tags(key) + ', '
print ('keyword : ', keyword)
print ('loc : ', loc)

# rec type
rec = ''
apply_qual = remove_tags(html_job.find('dl', {'class':'tbList'})).replace('\n','')
apply_qual1 = apply_qual.find('경력')
apply_qual2 = apply_qual.find('학력')
if apply_qual2 >= 0:
    rec = apply_qual[apply_qual1+2:apply_qual2].replace(' ','')
else:
    rec = apply_qual[apply_qual1+2:].replace(' ','')
print ('rec type : ', rec)

title :  마이셀럽스 채용 - [마이셀럽스] 데이터 사이언티스트 정규직 채용 | 잡코리아
job :  데이터 사이언티스트
company :  (주)마이셀럽스
url :  https://www.jobkorea.co.kr/Recruit/GI_Read/38870020?Oem_Code=C1&logpath=1&stext=데이터 사이언티스트&listno=3
keyword :  IT분야 직무채용관, 빅데이터·AI(인공지능), 자연어처리(NLP), 데이터분석, 솔루션·SI·CRM·ERP, 모바일앱개발, 소프트웨어개발, 솔루션, 
loc :  서울 > 강남구
rec type :  경력(3년이상)


In [177]:
# 상세 채용 정보 내용
duties = ''
skills = ''
preference = ''
welfare = ''

content = html_iframe.find('div', {'class':'detailed-summary-contents'})
content_table = html_iframe.find('div', {'class':'recruitment'})
content_lines = content_table.find('div', {'class':'recruitment-item'})

ps = content.find_all('p')
for ip in ps:
    # duties
    str_ip = remove_tags(ip)
    if str(ip).find("<b>") > -1: # bold체로 구분한 경우
        case = -1
        if '담당업무' in str(ip): # duties
            case = 1
        elif any([needs in str(ip) for needs in ['필수', '자격']]): # skills
            case = 2
        elif '우대' in str(ip): # preference
            case = 3
        elif any([i in str(ip) for i in ['복지', '복리']]): # welfare
            case = 4
        else:
            case = -1
    if case == 1: # duties
        duties += str_ip + '\n'
    elif case == 2: # skills
        skills += str_ip + '\n'
    elif case == 3: # preference
        preference += str_ip + '\n'
    elif case == 4: # welfare
        welfare += str_ip + '\n'
        
print ('duties : \n',duties)
print ('skills : \n',skills)
print ('preference : \n',preference)
print ('welfare : \n',welfare)

duties : 
 담당업무
ㆍ자연어 기반 추천 서비스를 위한 알고리즘 모델링
ㆍ딥러닝 모델의 최적화/경량화 연구 및 개발
ㆍ콘텐츠 추천 시스템 연구 및 개발


skills : 
 자격요건
ㆍ학력 : 학력무관
ㆍ새로운 기술을 탐색하여 적용할 수 있는 분
ㆍ팀원과 소통하며 이슈를 공유하고 함께 연구/해결가능한 분
ㆍ자연어처리 혹은 이미지 기반 모델 생성 경험
ㆍTensorflow / Pytorch 등 딥러닝 라이브러리 숙달


preference : 
 모집분야 및 자격요건
우대사항
ㆍ머신러닝, 통계 관련 석사 / 박사
ㆍTransformer / BERT 등 NLP 딥러닝 모델 이해 또는 활용의 경험 여부
ㆍ딥러닝 모델의 최적화/경량화 연구 및 개발
ㆍML/AI 실무경력 3년 이상 혹은 이에 상응하는 역량
ㆍ인공지능 시스템을 상용 서비스로 운영해보신 분

제출서류
ㆍ포트폴리오 (필수사항 x)

복리후생
ㆍ연차 상관 없는 높은 연봉 상승율, 스톡옵션
ㆍ수평적인 문화 제도 (님/닉네임 사용)
ㆍ출퇴근 접근성이 좋은 강남 소재 사무실
ㆍ최고급 장비 지원 (맥북프로, 듀얼 모니터) 제공
ㆍ휴게실 및 음료 (차, 커피 등) 무제한 제공
ㆍ월별 업무관련 도서구입비 지원 및 세미나 참가비 지원
ㆍ생일휴가(유급) 및 선물 지급


전형절차
ㆍ서류전형 &gt; 면접 &gt; 최종합격
ㆍ면접일정은 추후 통보됩니다.


유의사항
ㆍ허위사실이 발견될 경우 채용이 취소될 수 있습니다.


welfare : 
 


In [174]:
welfare = ''
content = html_iframe.find('div', {'class':'detailed-summary-contents'})
print(content.find_all('p'))


print ('welfare : ',welfare)


#print (Data)

[<p class="heading"><i class="icon-header dev-case-type case1"></i>모집분야 및 자격요건</p>, <p><b><br/></b></p>, <p><b><br/></b></p>, <p><b><br/></b></p>, <p><b><br/></b></p>, <p><b><br/></b></p>, <p><b><br/></b></p>, <p><b><br/></b></p>, <p><b><br/></b></p>, <p><b><br/></b></p>, <p><b>데이터 사이언티스트</b> </p>, <p>( ○명 ) </p>, <p><b>담당업무</b></p>, <p>ㆍ자연어 기반 추천 서비스를 위한 알고리즘 모델링</p>, <p>ㆍ딥러닝 모델의 최적화/경량화 연구 및 개발</p>, <p>ㆍ콘텐츠 추천 시스템 연구 및 개발</p>, <p><br/></p>, <p><b>자격요건</b></p>, <p>ㆍ학력 : 학력무관</p>, <p>ㆍ새로운 기술을 탐색하여 적용할 수 있는 분</p>, <p>ㆍ팀원과 소통하며 이슈를 공유하고 함께 연구/해결가능한 분<br/></p>, <p>ㆍ자연어처리 혹은 이미지 기반 모델 생성 경험<br/></p>, <p>ㆍTensorflow / Pytorch 등 딥러닝 라이브러리 숙달</p>, <p><br/></p>, <p><b>우대사항</b></p>, <p>ㆍ머신러닝, 통계 관련 석사 / 박사</p>, <p>ㆍTransformer / BERT 등 NLP 딥러닝 모델 이해 또는 활용의 경험 여부</p>, <p>ㆍ딥러닝 모델의 최적화/경량화 연구 및 개발</p>, <p>ㆍML/AI 실무경력 3년 이상 혹은 이에 상응하는 역량</p>, <p>ㆍ인공지능 시스템을 상용 서비스로 운영해보신 분</p>, <p><br/></p>, <p><span style="font-weight: 700;">제출서류</span></p>, <p>ㆍ포트폴리오 (필수사항 x)</p>, <p><br/></p>, <p class="heading" 

In [169]:
# Data에 저장
length = Data.shape[0]
Data.loc[length] = [title, job, company, duties, skills, preference, welfare, loc, rec, url]
Data

,title,job,cname,duties,skills,preference,welfare,loc,rec type,url
0,카카오뱅크 채용 - [빅데이터분석] 데이터 사이언티스트 | 잡코리아,데이터 사이언티스트,카카오뱅크,담당업무\nㆍ머신러닝을 이용한 이상거래 탐지 모형 개발\nㆍ머신러닝을 활용한 FDS...,필수 경험과 역량\nㆍ머신러닝을 통한 업무 경험 또는 문제해결 경험이 3년 이상인 ...,우대사항\nㆍ금융사기 방지를 위한 이상탐지 분석 및 모형 개발 경험이 있는 분\nㆍ...,,경기 > 성남시 분당구,경력,https://www.jobkorea.co.kr/Recruit/GI_Read/390...
1,마이셀럽스 채용 - [마이셀럽스] 데이터 사이언티스트 정규직 채용 | 잡코리아,데이터 사이언티스트,(주)마이셀럽스,담당업무\nㆍ자연어 기반 추천 서비스를 위한 알고리즘 모델링\nㆍ딥러닝 모델의 최적...,,"우대사항\nㆍ머신러닝, 통계 관련 석사 / 박사\nㆍTransformer / BER...",,서울 > 강남구,경력(3년이상),https://www.jobkorea.co.kr/Recruit/GI_Read/388...
2,마이셀럽스 채용 - [마이셀럽스] 데이터 사이언티스트 정규직 채용 | 잡코리아,데이터 사이언티스트,(주)마이셀럽스,담당업무\nㆍ자연어 기반 추천 서비스를 위한 알고리즘 모델링\nㆍ딥러닝 모델의 최적...,자격요건\nㆍ학력 : 학력무관\nㆍ새로운 기술을 탐색하여 적용할 수 있는 분\nㆍ팀...,"우대사항\nㆍ머신러닝, 통계 관련 석사 / 박사\nㆍTransformer / BER...",,서울 > 강남구,경력(3년이상),https://www.jobkorea.co.kr/Recruit/GI_Read/388...


In [164]:
print (url)

https://www.jobkorea.co.kr/Recruit/GI_Read/38870020?Oem_Code=C1&logpath=1&stext=데이터 사이언티스트&listno=3


In [148]:
# Data초기화
Data = pd.DataFrame({}, columns=[\
    'title','job', 'cname','duties','skills', 'preference', 'welfare', 'loc',\
    'rec type','url'])

In [111]:
driver.quit()